# **Prevendo o *churn* (abandono de clientes) de um banco**

## Descrição do notebook

O objetivo aqui é prever o churn (abandono de clientes) de um banco de dados fictício de uma instituição financeira. 

Para isso são fornecidos dois datasets: um dataset chamado *abandono_clientes* composto por 10000 linhas e 13 colunas de informação (features), sendo uma coluna “Exited” composta por dados binários: **1 se o cliente abandonou o banco**, **0 se não**.  

O segundo dataset possui 1000 linhas e 12 colunas e não possui a coluna “Exited”. 
Vamos prever essa coluna a partir do modelo de Machine Learning escolhido ao treinarmos sobre o conjunto de dados *abandono_clientes*.

Note que de acordo com o descrito acima temos um problema de **classificação binária**.


## (1) Importando a base de dados *abandono_clientes*

In [ ]:
import pandas as pd

In [ ]:
abandono_clientes = pd.read_csv(r"/content/Abandono_clientes.csv")

In [ ]:
abandono_clientes.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## (2) Análise exploratória dos dados

In [ ]:
# Informacoes dos nossos dados - Tipos de dados, nome das features, etc
abandono_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


* Temos dos dados acima que não há valores nulos no nosso dataframe. Se quisermos confirmar isso basta executarmos o código abaixo.

In [ ]:
# Verificando se ha valores nulos no nosso dataframe
abandono_clientes.isnull().sum() # Nao ha valores nulos

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
# Verificando as saidas de interesse
abandono_clientes["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

* Notamos que há um desbalanceamento entre o número de clientes que cancelam o cartão e o número de clientes que não. Logo, talvez seja interessante aplicarmos um *conceito de tratamento de dados desbalanceados*, como *estratificação*.

### (2.1) Análise das features (colunas) representativas do nosso dataframe

In [ ]:
abandono_clientes.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [ ]:
abandono_clientes.tail(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [ ]:
# Lista de paises
geography_list = abandono_clientes["Geography"].value_counts()
geography_list

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [ ]:
# Lista de generos
gender_list = abandono_clientes["Gender"].value_counts()
gender_list

Male      5457
Female    4543
Name: Gender, dtype: int64

* Como nosso trabalho aqui é selecionar as features que mais fazem sentido, é imediato ver que que as colunas "CustomerId", "Surname" e "Geography" não beneficiam a nossa análise e consequentemente não irão beneficiar nosso modelo de previsão, umas vez que não acrescentam informação relevante;

* Não eliminamos ainda a nossa coluna "Gender" pois é necessário uma análise da mesma para verificarmos se existe um número maior de clientes de algum genêro que cancela ou não cartões.

In [ ]:
# Verificando relacao entre Gender e Cancelamentos
import plotly.express as px

In [ ]:
px.histogram?

In [ ]:
fig = px.histogram(abandono_clientes, x="Exited", color=abandono_clientes["Gender"])
fig.show()

* Do plot acima vemos que a diferença entre os gêneros que cancelam ou não cartão é muito pequena, de modo que a coluna "Gender" pode não acrescentar um acréscimo relevante de precisão para o nosso modelo. Então a priori removemos também tal coluna.

## (3) Pré-processamento dos dados

In [ ]:
# Colunas a serem removidas do nosso dataframe  "CustomerId", "Surname","Geography" e "Gender"
abandono_clientes2 = abandono_clientes.copy()

In [ ]:
# Removendo as colunas
abandono_clientes2.drop(["CustomerId", "Surname","Geography", "Gender"], axis=1, inplace=True)

In [ ]:
# Guardando a coluna RowNumber
row_number = abandono_clientes2["RowNumber"]

In [ ]:
# Excluindo a coluna RowNumber a priori, ja que nao eh relevante para o nosso modelo
abandono_clientes2.drop(["RowNumber"], axis=1, inplace=True)

In [ ]:
# Check
abandono_clientes2.head(4)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0


In [ ]:
# Check
abandono_clientes2.shape

(10000, 9)

* Vamos avaliar a relação em ordem de grandeza entre os nossos dados.

In [ ]:
abandono_clientes2.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [ ]:
# Check dos tipos novamente
abandono_clientes2.dtypes

CreditScore          int64
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

* É possível vermos que a ordem de grandezas dos nossos dados são muito grandes entre si, então dependendo do algoritmo que formos utilizar para o treinamento do nosso modelo [(como algoritmos que usam métodos de distância)](https://medium.com/tentando-ser-um-unicórnio/porquê-e-quando-é-necessário-normalizar-os-dados-92e5cce445aa) talvez seja necessário *normalizarmos* os nossos dados. 

## (4) Modelos de Machine Learning (usando Train-Test-Split)

### (4.1) Preparando os dados que vamos usar

In [ ]:
# Vamos armazenar nossa coluna de labels em um array
import numpy as np

In [ ]:
# Armazenando os labels em um array
labels = np.array(abandono_clientes2["Exited"])

# Salvando a ordem das features
feature_list = list(abandono_clientes2.columns)

In [ ]:
# Removendo a coluna de labels do nosso dataframe
df = abandono_clientes2.drop("Exited", axis=1)

# Check
df.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [ ]:
# Convertendo nosso df para um array
data = np.array(df)

In [ ]:
# Normalizando os dados
from sklearn.preprocessing import StandardScaler

In [ ]:
data = StandardScaler().fit_transform(data)

In [ ]:
data

array([[-0.32622142,  0.29351742, -1.04175968, ...,  0.64609167,
         0.97024255,  0.02188649],
       [-0.44003595,  0.19816383, -1.38753759, ..., -1.54776799,
         0.97024255,  0.21653375],
       [-1.53679418,  0.29351742,  1.03290776, ...,  0.64609167,
        -1.03067011,  0.2406869 ],
       ...,
       [ 0.60498839, -0.27860412,  0.68712986, ..., -1.54776799,
         0.97024255, -1.00864308],
       [ 1.25683526,  0.29351742, -0.69598177, ...,  0.64609167,
        -1.03067011, -0.12523071],
       [ 1.46377078, -1.04143285, -0.35020386, ...,  0.64609167,
        -1.03067011, -1.07636976]])

### (4.2) Definindo nossa *baseline* (Modelo aleatório de comparação)

In [ ]:
# Importar train_test_split do scikitlearn 
from sklearn.model_selection import train_test_split

# Aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)

In [ ]:
# Criando baseline
baseline_preds = np.random.choice([0,1], size = len(test_labels))

print(baseline_preds)

[1 1 1 ... 1 0 0]


In [ ]:
# Importar biblioteca para calculo de métricas
from sklearn import metrics  

In [ ]:
print('\nClassification Report\n', metrics.classification_report(test_labels, baseline_preds)) 


Classification Report
               precision    recall  f1-score   support

           0       0.79      0.51      0.62      2003
           1       0.18      0.44      0.26       497

    accuracy                           0.50      2500
   macro avg       0.48      0.48      0.44      2500
weighted avg       0.67      0.50      0.55      2500



### (4.3) Modelo KNN

Vamos treinar nosso modelo usando o conjunto separado para a nossa baseline (sem usar por enquanto métodos mais avançados para modelos desbalanceados)

In [ ]:
# Importar o modelo de KNN
from sklearn.neighbors import KNeighborsClassifier 

# Treinando o modelo no conjunto de dados de treino
classifier = KNeighborsClassifier().fit(train_data, train_labels);

In [ ]:
# Aplicando o modelo treinado para a previsao em todo o conjunto de teste
predictions_labels = classifier.predict(test_data)

In [ ]:
# Exibindo dataframe com valores 10 reais e suas respectivas previsoes
p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions_labels})  
p.head(10)

,Real,Previsto
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,1,0
8,0,0
9,0,0


In [ ]:
print('\nClassification Report\n', metrics.classification_report(test_labels, predictions_labels)) 


Classification Report
               precision    recall  f1-score   support

           0       0.87      0.95      0.91      2003
           1       0.67      0.42      0.52       497

    accuracy                           0.84      2500
   macro avg       0.77      0.69      0.71      2500
weighted avg       0.83      0.84      0.83      2500



* Podemos ver que nossa métrica *f1-score* não desempenhou tão bem o seu papel, que era prever os clientes que cancelam os cartões. As outras métricas também não foram tão boas.

### (4.4) Modelo SVM

In [ ]:
# Importar o modelo SVM
from sklearn.svm import SVC

# Instanciacao e determinacao dos hiperparametros do SVM: tipo de kernel
classifier2 = SVC(kernel='rbf')

# Treinando o SVM
classifier2.fit(train_data,train_labels)

SVC()

In [ ]:
# Aplicando o modelo treinado para a previsao em todo o conjunto de teste
predictions2_labels = classifier2.predict(test_data)

In [ ]:
print('\nClassification Report\n', metrics.classification_report(test_labels, predictions2_labels)) 


Classification Report
               precision    recall  f1-score   support

           0       0.86      0.98      0.91      2003
           1       0.80      0.36      0.49       497

    accuracy                           0.85      2500
   macro avg       0.83      0.67      0.70      2500
weighted avg       0.85      0.85      0.83      2500



### (4.5) Modelo Random Forest

In [ ]:
# Importando o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# Treinando o modelo 
classifier3 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data, train_labels);

In [ ]:
# Aplicando o modelo treinado para a previsão do resultado do teste
predictions3_labels = classifier3.predict(test_data)

In [ ]:
print('\nClassification Report\n', metrics.classification_report(test_labels, predictions3_labels)) 


Classification Report
               precision    recall  f1-score   support

           0       0.87      0.96      0.91      2003
           1       0.70      0.40      0.51       497

    accuracy                           0.85      2500
   macro avg       0.78      0.68      0.71      2500
weighted avg       0.83      0.85      0.83      2500



### (4.6) Conclusão usando Train-Test-Split

Usando esta divisão para conjunto de treino e teste temos que os modelos de classificação usados não desempenham um bom papel, o que era de se esperar. 

Vamos analisar agora os modelos usando o método de **Validação Cruzada**.

## (5) Modelos de Machine Learning (usando Validação Cruzada)

### (5.1) Random Forest com Validação Cruzada

In [ ]:
# Random Forest com validacao cruzada (cv)
from sklearn.model_selection import cross_val_score

classifier_cv = RandomForestClassifier(n_estimators= 10, random_state=42)

In [ ]:
for num_cv in [5, 10, 20, 30]:
  print("cv = ", num_cv)
  scores_cv = cross_val_score(classifier_cv, data, labels, cv=num_cv)
  scores_cv_precision = cross_val_score(classifier_cv, data, labels, cv=num_cv, scoring='precision')
  scores_cv_recall = cross_val_score(classifier_cv, data, labels, cv=num_cv, scoring='recall')
  scores_cv_f1 = cross_val_score(classifier_cv, data, labels, cv=num_cv, scoring='f1')

  print("Acurácia: %0.2f (+/- %0.2f)" % (scores_cv.mean(), scores_cv.std() * 2))
  print("Precision: %0.2f (+/- %0.2f)" % (scores_cv_precision.mean(), scores_cv_precision.std() * 2))
  print("Recall: %0.2f (+/- %0.2f)" % (scores_cv_recall.mean(), scores_cv_recall.std() * 2))
  print("F1: %0.2f (+/- %0.2f)" % (scores_cv_f1.mean(), scores_cv_f1.std() * 2))

cv =  5
Acurácia: 0.85 (+/- 0.01)
Precision: 0.73 (+/- 0.03)
Recall: 0.40 (+/- 0.05)
F1: 0.52 (+/- 0.04)
cv =  10
Acurácia: 0.85 (+/- 0.02)
Precision: 0.73 (+/- 0.06)
Recall: 0.42 (+/- 0.10)
F1: 0.53 (+/- 0.09)
cv =  20
Acurácia: 0.84 (+/- 0.02)
Precision: 0.71 (+/- 0.06)
Recall: 0.40 (+/- 0.10)
F1: 0.51 (+/- 0.09)
cv =  30
Acurácia: 0.85 (+/- 0.03)
Precision: 0.71 (+/- 0.10)
Recall: 0.41 (+/- 0.12)
F1: 0.52 (+/- 0.12)


### (5.2) Modelo SVM com Validação Cruzada

In [ ]:
# Instanciacao e determinacao dos hiperparâmetros do SVM: tipo de kernel
classifierSVC_cv = SVC(kernel='rbf')

In [ ]:
for num_cv in [5, 10, 20, 30]:
  print("cv = ", num_cv)
  print(" ")
  scoresSVC_cv = cross_val_score(classifierSVC_cv, data, labels, cv=num_cv)
  scoresSVC_cv_precision = cross_val_score(classifierSVC_cv, data, labels, cv=num_cv, scoring='precision')
  scoresSVC_cv_recall = cross_val_score(classifierSVC_cv, data, labels, cv=num_cv, scoring='recall')
  scoresSVC_cv_f1 = cross_val_score(classifierSVC_cv, data, labels, cv=num_cv, scoring='f1')

  print("Acurácia: %0.2f (+/- %0.2f)" % (scoresSVC_cv.mean(), scoresSVC_cv.std() * 2))
  print("Precision: %0.2f (+/- %0.2f)" % (scoresSVC_cv_precision.mean(), scoresSVC_cv_precision.std() * 2))
  print("Recall: %0.2f (+/- %0.2f)" % (scoresSVC_cv_recall.mean(), scoresSVC_cv_recall.std() * 2))
  print("F1: %0.2f (+/- %0.2f)" % (scoresSVC_cv_f1.mean(), scoresSVC_cv_f1.std() * 2))

cv =  5
 
Acurácia: 0.85 (+/- 0.01)
Precision: 0.81 (+/- 0.06)
Recall: 0.36 (+/- 0.04)
F1: 0.50 (+/- 0.04)
cv =  10
 
Acurácia: 0.85 (+/- 0.02)
Precision: 0.81 (+/- 0.08)
Recall: 0.36 (+/- 0.08)
F1: 0.50 (+/- 0.08)
cv =  20
 
Acurácia: 0.85 (+/- 0.02)
Precision: 0.81 (+/- 0.11)
Recall: 0.36 (+/- 0.10)
F1: 0.50 (+/- 0.10)
cv =  30
 
Acurácia: 0.85 (+/- 0.03)
Precision: 0.81 (+/- 0.13)
Recall: 0.36 (+/- 0.12)
F1: 0.50 (+/- 0.13)


### (5.3) Conclusões sobre os modelos com a aplicação da Validação Cruzada

Como em geral o Random Forest é um modelo de classificação bom, uma vez também que é baseado em Ensemble, vamos escolher tal modelo como sendo nosso modelo de classificação (com **cv=30**), uma vez que se comparado aos outros modelos analisados aqui ele é o com melhores métricas.

Tal escolha se faz principalmente pelo fato do tempo para uma análise ainda mais profunda (com técnicas de *sintonia de hiperparâmetros*, por exemplo) ser limitado.

## (6) Implementação do nosso modelo Random Forest com Validação Cruzada

(Não vamos normalizar nossos dados uma vez que escolhemos o modelo Random Forest.)

In [ ]:
# Ajustando o modelo Random Forest escolhido
classifier_cv.fit(data, labels)

RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
# Importando a base de dados de teste
dados_teste = pd.read_csv("/content/Abandono_teste.csv", sep=";")

In [ ]:
dados_teste.head(4)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,10001,15798485,Copley,565,France,Male,31,1,0.00,1,0,1,20443.08
1,10002,15588959,T'ang,569,France,Male,34,4,0.00,1,0,1,4045.90
2,10003,15624896,Ku,669,France,Female,20,7,0.00,2,1,0,128838.67
3,10004,15639629,McConnan,694,France,Male,39,4,173255.48,1,1,1,81293.10


In [ ]:
dados_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        1000 non-null   int64  
 1   CustomerId       1000 non-null   int64  
 2   Surname          1000 non-null   object 
 3   CreditScore      1000 non-null   int64  
 4   Geography        1000 non-null   object 
 5   Gender           1000 non-null   object 
 6   Age              1000 non-null   int64  
 7   Tenure           1000 non-null   int64  
 8   Balance          1000 non-null   float64
 9   NumOfProducts    1000 non-null   int64  
 10  HasCrCard        1000 non-null   int64  
 11  IsActiveMember   1000 non-null   int64  
 12  EstimatedSalary  1000 non-null   float64
dtypes: float64(2), int64(8), object(3)
memory usage: 101.7+ KB


In [ ]:
def ajusta_dados(data_set_csv):
  ''' 
  data_set: arquivo csv com as mesmas features do problema em questao

  retorno: retorna um array do dataset filtrado
  '''
  # Importa a base de dados
  data_set = pd.read_csv(data_set_csv, sep=";")

  # Remove as colunas sem muito a oferecer
  data_set.drop(["RowNumber", "CustomerId", "Surname","Geography", "Gender"], axis=1, inplace=True)

  # Salvando a ordem das features
  feature_list = list(data_set.columns)

  # Convertendo nosso df para um array
  data = np.array(data_set)

  return data

In [ ]:
data_teste = ajusta_dados("/content/Abandono_teste.csv")

In [ ]:
data_teste.shape

(1000, 8)

In [ ]:
prediction_labels = classifier_cv.predict(data_teste)

(1000,)

In [ ]:
prediction_labels.shape

(1000,)

In [ ]:
# Exibindo dataframe com valores 10 reais e suas respectivas previsoes
row_numbers = np.array(dados_teste["RowNumber"])

In [ ]:
row_numbers.shape

(1000,)

In [ ]:
labels_teste = pd.DataFrame({'RowNumber': row_numbers, 'Previsto': prediction_labels})  
labels_teste.head(10)

,RowNumber,Previsto
0,10001,0
1,10002,0
2,10003,1
3,10004,0
4,10005,0
5,10006,1
6,10007,0
7,10008,0
8,10009,1
9,10010,1


In [ ]:
labels_teste.to_excel("labels_teste.xlsx")